In [18]:
import pandas as pd
from datetime import datetime
from pytz import timezone, all_timezones
import json

In [19]:
# Create DF from CSV file
df = pd.read_csv('data/gps_with_location_info.csv')
df = df.drop(['Unnamed: 0'], axis=1) #drop index column
df['time'] = pd.to_datetime(df['time']) #change 'time' column type to datetime64 from python object

df.head()

,track,time,alt,lat,long,city,state,country
0,0,2019-05-08 06:57:15+00:00,741.08,36.111288,-115.261516,Spring Valley,Nevada,US
1,0,2019-05-08 15:27:00+00:00,754.07,36.111416,-115.261345,Spring Valley,Nevada,US
2,0,2019-05-08 15:27:08+00:00,753.00,36.111426,-115.261353,Spring Valley,Nevada,US
3,0,2019-05-08 15:27:09+00:00,753.00,36.111426,-115.261353,Spring Valley,Nevada,US
4,0,2019-05-08 15:28:09+00:00,750.00,36.111330,-115.261409,Spring Valley,Nevada,US


In [43]:
country_timezones = json.load(open('data/country_timezones.py', 'r'))

In [44]:
country_timezones['countries']

[{'timezones': ['Europe/Andorra'],
  'code': 'AD',
  'continent': 'Europe',
  'name': 'Andorra',
  'capital': 'Andorra la Vella'},
 {'timezones': ['Asia/Kabul'],
  'code': 'AF',
  'continent': 'Asia',
  'name': 'Afghanistan',
  'capital': 'Kabul'},
 {'timezones': ['America/Antigua'],
  'code': 'AG',
  'continent': 'North America',
  'name': 'Antigua and Barbuda',
  'capital': "St. John's"},
 {'timezones': ['Europe/Tirane'],
  'code': 'AL',
  'continent': 'Europe',
  'name': 'Albania',
  'capital': 'Tirana'},
 {'timezones': ['Asia/Yerevan'],
  'code': 'AM',
  'continent': 'Asia',
  'name': 'Armenia',
  'capital': 'Yerevan'},
 {'timezones': ['Africa/Luanda'],
  'code': 'AO',
  'continent': 'Africa',
  'name': 'Angola',
  'capital': 'Luanda'},
 {'timezones': ['America/Argentina/Buenos_Aires',
   'America/Argentina/Cordoba',
   'America/Argentina/Jujuy',
   'America/Argentina/Tucuman',
   'America/Argentina/Catamarca',
   'America/Argentina/La_Rioja',
   'America/Argentina/San_Juan',
   'A

In [3]:
# Clean incorrect MY data - replace with SG
# https://kanoki.org/2019/07/17/pandas-how-to-replace-values-based-on-conditions/
df.loc[(df['country'] == 'MY'),'city']='Singapore'
df.loc[(df['country'] == 'MY'),'state']='NaN'
df.loc[(df['country'] == 'MY'),'country']='SG'

In [11]:
countries = df['country'].unique()
countries

array(['US', 'CA', 'DE', 'ES', 'IT', 'AT', 'LI', 'CH', 'FR', 'MA', 'PT',
       'BE', 'DK', 'NZ', 'AU', 'SG', 'PH', 'VN', 'TH'], dtype=object)

In [15]:
countryMatch = {}
for c in countries:
    match = input(c)
    countryMatch[c] = match.replace(" ", "_")

#  'US': 'US',
#  'CA': 'Canada',
#  'DE': 'Germany',
#  'ES': 'Spain',
#  'IT': 'Italy',
#  'AT': 'Austria',
#  'LI': 'Liechtenstein',
#  'CH': 'Switzerland',
#  'FR': 'France',
#  'MA': 'Morocco',
#  'PT': 'Portugal',
#  'BE': 'Belgium',
#  'DK': 'Denmark',
#  'NZ': 'New_Zealand',
#  'AU': 'Australia',
#  'SG': 'Singapore',
#  'PH': 'Philippines',
#  'VN': 'Vietnam',
#  'TH': 'Thailand'

USUS
CACanada
DEGermany
ESSpain
ITItaly
ATAustria
LILiechtenstein
CHSwitzerland
FRFrance
MAMorocco
PTPortugal
BEBelgium
DKDenmark
NZNew Zealand
AUAustralia
SGSingapore
PHPhilippines
VNVietnam
THThailand


In [16]:
countryMatch

{'US': 'US',
 'CA': 'Canada',
 'DE': 'Germany',
 'ES': 'Spain',
 'IT': 'Italy',
 'AT': 'Austria',
 'LI': 'Liechtenstein',
 'CH': 'Switzerland',
 'FR': 'France',
 'MA': 'Morocco',
 'PT': 'Portugal',
 'BE': 'Belgium',
 'DK': 'Denmark',
 'NZ': 'New_Zealand',
 'AU': 'Australia',
 'SG': 'Singapore',
 'PH': 'Philippines',
 'VN': 'Vietnam',
 'TH': 'Thailand'}

In [5]:

# https://stackoverflow.com/questions/48979604/pandas-for-each-unique-value-in-one-column-get-unique-values-in-another-column
states = df.groupby('state').apply(lambda x: x['country'].unique())

# len(df['country'].unique())
# type(dfState)
states

state
Aargau             [CH]
Alberta            [CA]
Andalusia          [ES]
Arizona            [US]
Arkansas           [US]
                   ... 
Western Visayas    [PH]
Wisconsin          [US]
Zealand            [DK]
Zug                [CH]
Zurich             [CH]
Length: 133, dtype: object

In [ ]:
# type(df['time'][0])
df['time'][0].to_pydatetime()

In [10]:
# https://www.saltycrane.com/blog/2009/05/converting-time-zones-datetime-objects-python/
for zone in all_timezones:
        print(zone)
#     if 'Australia' in zone:
#         print (zone)

Africa/Abidjan
Africa/Accra
Africa/Addis_Ababa
Africa/Algiers
Africa/Asmara
Africa/Asmera
Africa/Bamako
Africa/Bangui
Africa/Banjul
Africa/Bissau
Africa/Blantyre
Africa/Brazzaville
Africa/Bujumbura
Africa/Cairo
Africa/Casablanca
Africa/Ceuta
Africa/Conakry
Africa/Dakar
Africa/Dar_es_Salaam
Africa/Djibouti
Africa/Douala
Africa/El_Aaiun
Africa/Freetown
Africa/Gaborone
Africa/Harare
Africa/Johannesburg
Africa/Juba
Africa/Kampala
Africa/Khartoum
Africa/Kigali
Africa/Kinshasa
Africa/Lagos
Africa/Libreville
Africa/Lome
Africa/Luanda
Africa/Lubumbashi
Africa/Lusaka
Africa/Malabo
Africa/Maputo
Africa/Maseru
Africa/Mbabane
Africa/Mogadishu
Africa/Monrovia
Africa/Nairobi
Africa/Ndjamena
Africa/Niamey
Africa/Nouakchott
Africa/Ouagadougou
Africa/Porto-Novo
Africa/Sao_Tome
Africa/Timbuktu
Africa/Tripoli
Africa/Tunis
Africa/Windhoek
America/Adak
America/Anchorage
America/Anguilla
America/Antigua
America/Araguaina
America/Argentina/Buenos_Aires
America/Argentina/Catamarca
America/Argentina/ComodRivad

In [ ]:
df.loc[df['country'] == 'MY']

In [ ]:
# df['time'] = pd.to_datetime(df['time'])
# df['time']

import datetime as dt
# readable = datetime.datetime.fromtimestamp().isoformat()
# print(readable)

startdate = dt.datetime.combine(df['time'][0].date(), df['time'][0].time())
for t in df['time']:
    print(t.tzinfo)

In [ ]:
# Create new DF of values that need to be tracked
dfValues = df[['track','time','alt','lat','long']].copy()
dfValues

In [ ]:
# Find difference between trailing rows
dfDiff = dfValues.diff()
dfDiff.head()

In [ ]:
dfValues = dfValues.diff()

In [ ]:
# Create new Df with only values that need to be differentiated
dfValues = df[['track','alt','lat','long']].copy()
dfValues.dtypes

In [ ]:
type(df['time'][0])

In [ ]:
old[['A', 'C', 'D']].copy()

In [ ]:
# Merge Address DF with original DF
dfFinal = df.merge(dfAddress, left_index=True, right_index=True)


In [ ]:

dfTimeSortedDiff = dfTimeSorted.diff()
dfTimeSortedDiff.head()

In [ ]:
# Merge DFs and rename columns
dff = dfTimeSorted.merge(dfTimeSortedDiff, left_index=True, right_index=True)
dff.columns = ['TrackNo','Time', 'Latitude', 'Longitude','Elevation','NewTrack','TimeDiff','LatitudeDiff', 'LongitudeDiff', 'ElevationDiff']
dff['NewTrack'] = dff['NewTrack'].fillna(1).astype(int)
dff.head()

In [ ]:
dff.loc[(dff['LatitudeDiff'] < 0.0009) & (dff['LatitudeDiff'] > -0.0009) & (dff['LongitudeDiff'] < 0.0009) & (dff['LongitudeDiff'] > -0.0009)]

In [ ]:
dff['NewTrack'] = dff['NewTrack'].map({'0': 'NaN'})

In [ ]:
# Search for the coordinates and unpack resulting address dictionary
addy = json.loads(json.dumps(rg.search(coordAU)[0]))